In [12]:
from pyupbit import *
import pandas
import numpy as np

# Volatality Breakout BackTest
using upbit data 21-6-22~22-1-7 (200days)

w/o fee & slippage 1.1

w/ fee & slippage 0.8

93 transactions made but ony 39 Earns (54 lose)


In [38]:
fee = 0.99700225

df = get_ohlcv("KRW-BTC")
#print(df)

df['diff'] = (df['high']-df['low'])*0.5
df['diff_shift1'] = df['diff'].shift(1)
df['target'] = df['open']+df['diff_shift1']

df['profit'] = np.where(df['high']>df['target'],df['close']/df['target']*fee,1)
profit = df['profit'].cumprod()[-2]

print('profit: ',profit)
print('# Transactions: ',len(df[df.profit!=1]),'\n# Earn: ',len(df[df.profit>1]),'\n# Lose: ',len(df[df.profit<1]))
# df[['open','close','high','target','profit']]
# print(df[df.profit>1].profit.cumprod()[-1])

profit:  0.831354446561785
# Transactions:  93 
# Earn:  39 
# Lose:  54
